In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error

In [11]:
def read_yellow_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration']=df.tpep_dropoff_datetime-df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60 )
    df = df [(df.duration >= 1) & (df.duration <= 60)]
    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_y01_plain = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
df_y01 = read_yellow_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_y02 = read_yellow_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [10]:
# Question 1
len(df_y01_plain.columns)

19

In [12]:
# Question 2
df_y01_plain['duration']=df_y01_plain.tpep_dropoff_datetime-df_y01_plain.tpep_pickup_datetime
df_y01_plain.duration = df_y01_plain.duration.apply(lambda td: td.total_seconds()/60 )
df_y01_plain.duration.std()

46.44530513776499

In [14]:
# Question 3
round(((df_y01_plain.duration >= 1) & (df_y01_plain.duration <= 60)).mean()*100)

98

In [15]:
# Question 4
dv_y = DictVectorizer()
categorical = ["PULocationID", "DOLocationID"]
train_01_dicts = df_y01[categorical].to_dict(orient='records')
X_01_train = dv_y.fit_transform(train_01_dicts)
y_01_train = df_y01.duration.values
X_01_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [16]:
# Question 5
lr_y = LinearRegression()
lr_y.fit(X_01_train,y_01_train)

y_01_pred = lr_y.predict(X_01_train)

mean_squared_error(y_01_train,y_01_pred, squared=False)

6.986190135971174

In [18]:
# Question 6
val_02_dicts = df_y02[categorical].to_dict(orient='records')
X_02_val = dv_y.transform(val_02_dicts)
y_02_val = df_y02.duration.values
y_02_pred = lr_y.predict(X_02_val)

mean_squared_error(y_02_val,y_02_pred, squared=False)

7.786389483040939